### Importing pyspark

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.1.1-bin-hadoop3.2')

In [2]:
from pyspark.sql import SparkSession

In [3]:
#Initializing Spark Session
spark = SparkSession.builder.appName('nlpproject').getOrCreate()

### Importing data

In [6]:
#Reading Data
data = spark.read.csv('SMSSpamCollection',sep='\t')

In [8]:
data.show(5)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
+----+--------------------+
only showing top 5 rows



## Data Preprocessing

### Renaming Columns

In [9]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [11]:
data.show(5)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



### Adding Length Column

In [13]:
from pyspark.sql.functions import length

In [14]:
data = data.withColumn('length',length(data['text']))

In [15]:
data.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



#### Comparing whether the length of ham messages are different from spam messages

In [16]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



### NLP Operations

In [17]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer

In [24]:
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf = IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

## Model Building

In [25]:
from pyspark.ml.feature import VectorAssembler

In [26]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

#### Using Pipeline cleaning the text and making it ready for model 

In [29]:
from pyspark.ml import Pipeline

In [30]:
#Preparing for the above steps
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,count_vec,idf,clean_up])

In [31]:
cleaner = data_prep_pipe.fit(data)

In [32]:
clean_data = cleaner.transform(data)

In [34]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [35]:
final_data = clean_data.select(['label','features'])

In [36]:
final_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
+-----+--------------------+
only showing top 5 rows



#### Train Test Split

In [37]:
train_data , test_data = final_data.randomSplit([0.7,0.3],101)

### Naive Bayes Classifier

In [27]:
from pyspark.ml.classification import NaiveBayes

In [28]:
nb = NaiveBayes()

In [38]:
spam_detector = nb.fit(train_data)

#### Test Results

In [39]:
test_results = spam_detector.transform(test_data)

In [40]:
test_results.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,4,50,...|[-828.94645027671...|[1.0,1.4407467584...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-999.15650190371...|[1.0,6.5948418058...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1379.6441477676...|[1.0,3.0567822047...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-216.91825807345...|[1.0,1.8562786945...|       0.0|
|  0.0|(13424,[0,1,17,19...|[-797.51858134302...|[1.0,5.3544747914...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



### Evaluation Metrics

In [41]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [42]:
acc_eval = MulticlassClassificationEvaluator()

In [43]:
acc = acc_eval.evaluate(test_results)

In [44]:
print('Accuracy of model')
print(acc)

Accuracy of model
0.9297540456835104
